In [ ]:
import logging
import warnings

logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

warnings.filterwarnings("ignore")

from freamon import Freamon

In [2]:
view_gen, runtimes = \
  Freamon().pipeline_from_py_file('pipelines--mlinspect--credit.py', cmd_args=[])

INFO:root:Patching sys.argv with ['eyes']
INFO:root:Executing instrumented user pipeline with mlinspect
INFO:root:Redirecting the pipeline's stdout to pipeline-output.txt
INFO:root:---RUNTIME: Instrumented execution took 4251.723051071167 ms
INFO:root:Registering test source 1 with columns: ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income-per-year', 'mlinspect_lineage']


In [3]:
df = view_gen.test_view(sliceable_by=['race', 'sex'], with_features=False, with_y_true=True, with_y_pred=True)

INFO:root:
SELECT fs1.race, fs1.sex, ftv.y_true, ftv.y_pred
FROM _freamon_test_view ftv
JOIN _freamon_source_1_with_prov_view fs1  ON fs1.prov_id_source_1 = ftv.prov_id_source_1
   
        


# Group fairness as aggregation query

In [4]:
view_gen.db.execute(
"""
SELECT 
    (race!='Black' OR sex!='Male') AS privileged, 
    SUM(CAST((y_true=1 AND y_pred=1) AS INTEGER)) AS true_positive,
    SUM(CAST((y_true=1 AND y_pred=0) AS INTEGER)) AS false_negative,    
    SUM(CAST((y_true=0 AND y_pred=1) AS INTEGER)) AS false_positive,    
    SUM(CAST((y_true=0 AND y_pred=0) AS INTEGER)) AS true_negative,    
FROM df
GROUP BY privileged
"""
).df()

,privileged,true_positive,false_negative,false_positive,true_negative
0,True,192.0,235.0,126.0,1437.0
1,False,9.0,7.0,18.0,104.0


In [12]:
view_gen.db.execute(
"""
SELECT 
    race='Black' AS black,
    sex='Male' AS male,
    SUM(CAST((y_true=1 AND y_pred=1) AS INTEGER)) AS true_positive,
    SUM(CAST((y_true=1 AND y_pred=0) AS INTEGER)) AS false_negative,    
    SUM(CAST((y_true=0 AND y_pred=1) AS INTEGER)) AS false_positive,    
    SUM(CAST((y_true=0 AND y_pred=0) AS INTEGER)) AS true_negative,    
FROM df
GROUP BY GROUPING SETS ((race='Black', sex='Male'), (race='Black'), (sex='Male'))
"""
).df()



,black,male,true_positive,false_negative,false_positive,true_negative
0,False,False,23.0,51.0,38.0,590.0
1,True,True,9.0,7.0,18.0,104.0
2,False,True,164.0,180.0,77.0,724.0
3,True,False,5.0,4.0,11.0,123.0
4,False,NaN,187.0,231.0,115.0,1314.0
5,True,NaN,14.0,11.0,29.0,227.0
6,NaN,False,28.0,55.0,49.0,713.0
7,NaN,True,173.0,187.0,95.0,828.0
8,NaN,NaN,201.0,242.0,144.0,1541.0


# Slicefinder

In [13]:
view_gen.db.execute(
"""
SELECT 
    race='Black' AS black,
    sex='Male' AS male,
    AVG(-(y_true * log(y_pred_proba) + (1 - y_true) * log(1.0 - y_pred_proba))) AS avg_logloss,
    VARIANCE(-(y_true * log(y_pred_proba) + (1 - y_true) * log(1.0 - y_pred_proba))) AS var_logloss,    
    COUNT(*) as size
    
FROM (SELECT race, sex, y_true, IF(y_pred=0, 0.01, 0.99) AS y_pred_proba FROM df)
GROUP BY GROUPING SETS ((race='Black', sex='Male'), (race='Black'), (sex='Male'))
"""
).df()

,black,male,avg_logloss,var_logloss,size
0,False,False,0.257373,0.441527,702
1,True,True,0.365893,0.595088,138
2,False,True,0.452293,0.693868,1145
3,True,False,0.213697,0.376564,143
4,False,NaN,0.378209,0.606625,1847
5,True,NaN,0.288441,0.487950,281
6,NaN,False,0.249981,0.430343,845
7,NaN,True,0.443000,0.683488,1283
